# Emotion Recognition Using Convoluted Neural Network

By: Aurelius Justin Lim

This notebook aims to create a CNN in order to recognition human emotions. The emotions to be experimented are anger, happiness, and sadness. To do this, the network is trained from a dataset taken from kaggle. A training, test and validation set is used to ensure the correctness and optimal performance of the model. Ultimately, the model created in the notebook is integrated into an application that allows users to upload a photo and allow the model to determine the illustrated emotion. 

**Human Face Emotions By SANIDHYAK**

Reference: https://www.kaggle.com/datasets/sanidhyak/human-face-emotions?resource=download